In [1]:
!pip install squarify

In [0]:
import re
import string
from collections import Counter
import squarify
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import spacy
from spacy.tokenizer import Tokenizer

from bs4 import BeautifulSoup
import html as ihtml

import requests
import sqlite3

count_df = pd.DataFrame(columns=['job_count', 'title_name', 'city_name'])
global count_df

In [0]:
def extract_job_title(soup):
    jobs = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        for a in div.find_all(name="a", attrs={"data-tn-element":"jobTitle"}):
            jobs.append(a["title"])
    return(jobs)

In [0]:
def extract_company(soup):
    companies = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        company = div.find_all(name="span", attrs={"class":"company"})
        if len(company) > 0:
            for b in company:
                companies.append(b.text.strip())
        else:
            sec_try = div.find_all(name="span", attrs={"class":"result-linl-source"})
            for span in sec_try:
                companies.append(span.text.strip())
    return(companies)

In [0]:
def extract_location(soup):
    locations = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        try:
            locations.append(div.find(name="span", attrs={"class":"location"}).text)
        except:
            locations.append("None")
    return(locations)

In [0]:
def extract_salary(soup):
    salaries = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        try:
            salaries.append(div.find(name="span", attrs={"class":"salaryText"}).text.replace("\n",""))
        except:
            salaries.append("None")
    return(salaries)

In [0]:
def extract_job_title(soup):
    jobs = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        for a in div.find_all(name="a", attrs={"data-tn-element":"jobTitle"}):
            jobs.append(a["title"])
    return(jobs)

In [0]:
def extract_url(soup):
    urls = []
    for div1 in soup.find_all(name="div", attrs={"class":"row"}):
        for div2 in div1.find_all(name="div", attrs={"class":"title"}):
            for a in div2.find_all(name="a", href=True):
                urls.append(a['href'])
    return(urls)

def extract_desc(urls):
    descs = []
    for url in urls:
        full_url = "https://www.indeed.com" + url
        detail_page = requests.get(full_url)
        detail_soup = BeautifulSoup(detail_page.text, "html.parser")
        
        for div in detail_soup.find_all(name="div", attrs={"id":"jobDescriptionText"}):
            descs.append(div.text)
    return(descs)

In [0]:
def extract_count(soup):
    for div in soup.find_all(name="div", attrs={"id":"searchCountPages"}):
        temp_str = div.text.replace(" ", "")
        temp_count_str = re.search('of(.*)jobs', temp_str)
        count_str = re.sub('[^0-9]','', temp_count_str.group(1))
        count = int(count_str)
    return(count)

In [0]:
def extract_list(title_name, city_name, st_name):
    max_results = 100
    columns = ["city", "job_title", "company", "location", "salary", "description"]

    city_url = "https://www.indeed.com/jobs?q=" + title_name + \
               "&l=" + city_name + "%2C+" + st_name
    page = requests.get(city_url)
    soup = BeautifulSoup(page.text, "html.parser")
    max_counter = extract_count(soup)
    print(max_counter, title_name, city_name)
    t_name = title_name.replace('+', ' ')
    c_name = city_name.replace('+', ' ')
    count_df.append({'job_count': max_counter,
                                'title_name': t_name,
                                'city_name': c_name}, ignore_index=True)

    job_title_list = []
    company_list = []
    location_list = []
    salary_list = []
    desc_list = []

    for start in range(0, max_results, 10):
        city_url = "https://www.indeed.com/jobs?q=" + title_name + \
                   "&l=" + city_name + "%2C+" + st_name + \
                   "&start=" + str(start)
        page = requests.get(city_url)
        soup = BeautifulSoup(page.text, "html.parser")

        job_title_list.extend(extract_job_title(soup))
        company_list.extend(extract_company(soup))
        location_list.extend(extract_location(soup))
        salary_list.extend(extract_salary(soup))
        add_urls = extract_url(soup)
        desc_list.extend(extract_desc(add_urls))

    return job_title_list, company_list, location_list, salary_list, desc_list

def job_db(title, city, st):   
    j_list, c_list, l_list, s_list, d_list = extract_list(title, city, st)     
    temp_df = pd.DataFrame(list(zip(j_list, c_list, l_list, s_list, d_list)), 
                          columns = ['job_title', 'company', 'location', 'salary', 'description'])
    return temp_df

In [11]:
temp1_df = job_db('data+scientist', 'San+Jose', 'CA')
temp1_df['city'] = 'San Jose'
temp2_df = job_db('data+scientist', 'San+Francisco', 'CA')
temp2_df['city'] = 'San Francisco'
temp3_df = job_db('data+scientist', 'Seattle', 'WA')
temp3_df['city'] = 'Seattle'
temp4_df = job_db('data+scientist', 'Washington', 'DC')
temp4_df['city'] = 'Washington'
temp5_df = job_db('data+scientist', 'New+York', 'NY')
temp5_df['city'] = 'New York'
temp6_df = job_db('data+scientist', 'Baltimore', 'MD')
temp6_df['city'] = 'Baltimore'
temp7_df = job_db('data+scientist', 'Boulder', 'CO')
temp7_df['city'] = 'Boulder'
temp8_df = job_db('data+scientist', 'San+Diego', 'CA')
temp8_df['city'] = 'San Diego'
temp9_df = job_db('data+scientist', 'Denver', 'CO')
temp9_df['city'] = 'Denver'
temp10_df = job_db('data+scientist', 'Huntsville', 'AL')
temp10_df['city'] = 'Huntsville'
temp11_df = job_db('data+scientist', 'Colorado+Springs', 'CO')
temp11_df['city'] = 'Colorado Springs'
temp12_df = job_db('data+scientist', 'Houston', 'TX')
temp12_df['city'] = 'Houston'
temp13_df = job_db('data+scientist', 'Trenton', 'NJ')
temp13_df['city'] = 'Trenton'
temp14_df = job_db('data+scientist', 'Dallas', 'TX')
temp14_df['city'] = 'Dallas'
temp15_df = job_db('data+scientist', 'Columbus', 'OH')
temp15_df['city'] = 'Columbus'
temp16_df = job_db('data+scientist', 'Austin', 'TX')
temp16_df['city'] = 'Austin'
temp17_df = job_db('data+scientist', 'Philadelphia', 'PA')
temp17_df['city'] = 'Philadelphia'
temp18_df = job_db('data+scientist', 'Durham', 'NC')
temp18_df['city'] = 'Durham'
temp19_df = job_db('data+scientist', 'Raleigh', 'NC')
temp19_df['city'] = 'Raleigh'
temp20_df = job_db('data+scientist', 'Atlanta', 'GA')
temp20_df['city'] = 'Atlanta'

data_scientist_df = pd.concat([temp1_df, temp2_df, temp3_df, temp4_df, temp5_df,
                       temp6_df, temp7_df, temp8_df, temp9_df, temp10_df,
                       temp11_df, temp12_df, temp13_df, temp14_df, temp15_df,
                       temp16_df, temp17_df, temp18_df, temp19_df, temp20_df], ignore_index=True)

print(data_scientist_df.shape)

1315 data+scientist San+Jose
937 data+scientist San+Francisco
1106 data+scientist Seattle
1180 data+scientist Washington
980 data+scientist New+York
165 data+scientist Baltimore
161 data+scientist Boulder
164 data+scientist San+Diego
180 data+scientist Denver
22 data+scientist Huntsville
6 data+scientist Colorado+Springs
100 data+scientist Houston
91 data+scientist Trenton
261 data+scientist Dallas
74 data+scientist Columbus
185 data+scientist Austin
215 data+scientist Philadelphia
146 data+scientist Durham
148 data+scientist Raleigh
257 data+scientist Atlanta
(3350, 6)


In [12]:
temp1_df = job_db('web+developer', 'San+Jose', 'CA')
temp1_df['city'] = 'San Jose'
temp2_df = job_db('web+developer', 'San+Francisco', 'CA')
temp2_df['city'] = 'San Francisco'
temp3_df = job_db('web+developer', 'Seattle', 'WA')
temp3_df['city'] = 'Seattle'
temp4_df = job_db('web+developer', 'Washington', 'DC')
temp4_df['city'] = 'Washington'
temp5_df = job_db('web+developer', 'New+York', 'NY')
temp5_df['city'] = 'New York'
temp6_df = job_db('web+developer', 'Baltimore', 'MD')
temp6_df['city'] = 'Baltimore'
temp7_df = job_db('web+developer', 'Boulder', 'CO')
temp7_df['city'] = 'Boulder'
temp8_df = job_db('web+developer', 'San+Diego', 'CA')
temp8_df['city'] = 'San Diego'
temp9_df = job_db('web+developer', 'Denver', 'CO')
temp9_df['city'] = 'Denver'
temp10_df = job_db('web+developer', 'Huntsville', 'AL')
temp10_df['city'] = 'Huntsville'
temp11_df = job_db('web+developer', 'Colorado+Springs', 'CO')
temp11_df['city'] = 'Colorado Springs'
temp12_df = job_db('web+developer', 'Houston', 'TX')
temp12_df['city'] = 'Houston'
temp13_df = job_db('web+developer', 'Trenton', 'NJ')
temp13_df['city'] = 'Trenton'
temp14_df = job_db('web+developer', 'Dallas', 'TX')
temp14_df['city'] = 'Dallas'
temp15_df = job_db('web+developer', 'Columbus', 'OH')
temp15_df['city'] = 'Columbus'
temp16_df = job_db('web+developer', 'Austin', 'TX')
temp16_df['city'] = 'Austin'
temp17_df = job_db('web+developer', 'Philadelphia', 'PA')
temp17_df['city'] = 'Philadelphia'
temp18_df = job_db('web+developer', 'Durham', 'NC')
temp18_df['city'] = 'Durham'
temp19_df = job_db('web+developer', 'Raleigh', 'NC')
temp19_df['city'] = 'Raleigh'
temp20_df = job_db('web+developer', 'Atlanta', 'GA')
temp20_df['city'] = 'Atlanta'

web_developer_df = pd.concat([temp1_df, temp2_df, temp3_df, temp4_df, temp5_df,
                              temp6_df, temp7_df, temp8_df, temp9_df, temp10_df,
                              temp11_df, temp12_df, temp13_df, temp14_df, temp15_df,
                              temp16_df, temp17_df, temp18_df, temp19_df, temp20_df], ignore_index=True)
print(web_developer_df.shape)

1402 web+developer San+Jose
1414 web+developer San+Francisco
1422 web+developer Seattle
3439 web+developer Washington
1828 web+developer New+York
881 web+developer Baltimore
440 web+developer Boulder
384 web+developer San+Diego
597 web+developer Denver
79 web+developer Huntsville
68 web+developer Colorado+Springs
368 web+developer Houston
225 web+developer Trenton
779 web+developer Dallas
224 web+developer Columbus
1039 web+developer Austin
534 web+developer Philadelphia
377 web+developer Durham
360 web+developer Raleigh
617 web+developer Atlanta
(3631, 6)


In [13]:
temp1_df = job_db('ux+designer', 'San+Jose', 'CA')
temp1_df['city'] = 'San Jose'
temp2_df = job_db('ux+designer', 'San+Francisco', 'CA')
temp2_df['city'] = 'San Francisco'
temp3_df = job_db('ux+designer', 'Seattle', 'WA')
temp3_df['city'] = 'Seattle'
temp4_df = job_db('ux+designer', 'Washington', 'DC')
temp4_df['city'] = 'Washington'
temp5_df = job_db('ux+designer', 'New+York', 'NY')
temp5_df['city'] = 'New York'
temp6_df = job_db('ux+designer', 'Baltimore', 'MD')
temp6_df['city'] = 'Baltimore'
temp7_df = job_db('ux+designer', 'Boulder', 'CO')
temp7_df['city'] = 'Boulder'
temp8_df = job_db('ux+designer', 'San+Diego', 'CA')
temp8_df['city'] = 'San Diego'
temp9_df = job_db('ux+designer', 'Denver', 'CO')
temp9_df['city'] = 'Denver'
temp10_df = job_db('ux+designer', 'Huntsville', 'AL')
temp10_df['city'] = 'Huntsville'
temp11_df = job_db('ux+designer', 'Colorado+Springs', 'CO')
temp11_df['city'] = 'Colorado Springs'
temp12_df = job_db('ux+designer', 'Houston', 'TX')
temp12_df['city'] = 'Houston'
temp13_df = job_db('ux+designer', 'Trenton', 'NJ')
temp13_df['city'] = 'Trenton'
temp14_df = job_db('ux+designer', 'Dallas', 'TX')
temp14_df['city'] = 'Dallas'
temp15_df = job_db('ux+designer', 'Columbus', 'OH')
temp15_df['city'] = 'Columbus'
temp16_df = job_db('ux+designer', 'Austin', 'TX')
temp16_df['city'] = 'Austin'
temp17_df = job_db('ux+designer', 'Philadelphia', 'PA')
temp17_df['city'] = 'Philadelphia'
temp18_df = job_db('ux+designer', 'Durham', 'NC')
temp18_df['city'] = 'Durham'
temp19_df = job_db('ux+designer', 'Raleigh', 'NC')
temp19_df['city'] = 'Raleigh'
temp20_df = job_db('ux+designer', 'Atlanta', 'GA')
temp20_df['city'] = 'Atlanta'

ux_designer_df = pd.concat([temp1_df, temp2_df, temp3_df, temp4_df, temp5_df,
                            temp6_df, temp7_df, temp8_df, temp9_df, temp10_df,
                            temp11_df, temp12_df, temp13_df, temp14_df, temp15_df,
                            temp16_df, temp17_df, temp18_df, temp19_df, temp20_df], ignore_index=True)
print(ux_designer_df.shape)

634 ux+designer San+Jose
875 ux+designer San+Francisco
952 ux+designer Seattle
523 ux+designer Washington
868 ux+designer New+York
89 ux+designer Baltimore
132 ux+designer Boulder
107 ux+designer San+Diego
147 ux+designer Denver
8 ux+designer Huntsville
6 ux+designer Colorado+Springs
82 ux+designer Houston
41 ux+designer Trenton
178 ux+designer Dallas
47 ux+designer Columbus
274 ux+designer Austin
145 ux+designer Philadelphia
81 ux+designer Durham
81 ux+designer Raleigh
247 ux+designer Atlanta
(2997, 6)


In [14]:
temp1_df = job_db('ios+developer', 'San+Jose', 'CA')
temp1_df['city'] = 'San Jose'
temp2_df = job_db('ios+developer', 'San+Francisco', 'CA')
temp2_df['city'] = 'San Francisco'
temp3_df = job_db('ios+developer', 'Seattle', 'WA')
temp3_df['city'] = 'Seattle'
temp4_df = job_db('ios+developer', 'Washington', 'DC')
temp4_df['city'] = 'Washington'
temp5_df = job_db('ios+developer', 'New+York', 'NY')
temp5_df['city'] = 'New York'
temp6_df = job_db('ios+developer', 'Baltimore', 'MD')
temp6_df['city'] = 'Baltimore'
temp7_df = job_db('ios+developer', 'Boulder', 'CO')
temp7_df['city'] = 'Boulder'
temp8_df = job_db('ios+developer', 'San+Diego', 'CA')
temp8_df['city'] = 'San Diego'
temp9_df = job_db('ios+developer', 'Denver', 'CO')
temp9_df['city'] = 'Denver'
temp10_df = job_db('ios+developer', 'Huntsville', 'AL')
temp10_df['city'] = 'Huntsville'
temp11_df = job_db('ios+developer', 'Colorado+Springs', 'CO')
temp11_df['city'] = 'Colorado Springs'
temp12_df = job_db('ios+developer', 'Houston', 'TX')
temp12_df['city'] = 'Houston'
temp13_df = job_db('ios+developer', 'Trenton', 'NJ')
temp13_df['city'] = 'Trenton'
temp14_df = job_db('ios+developer', 'Dallas', 'TX')
temp14_df['city'] = 'Dallas'
temp15_df = job_db('ios+developer', 'Columbus', 'OH')
temp15_df['city'] = 'Columbus'
temp16_df = job_db('ios+developer', 'Austin', 'TX')
temp16_df['city'] = 'Austin'
temp17_df = job_db('ios+developer', 'Philadelphia', 'PA')
temp17_df['city'] = 'Philadelphia'
temp18_df = job_db('ios+developer', 'Durham', 'NC')
temp18_df['city'] = 'Durham'
temp19_df = job_db('ios+developer', 'Raleigh', 'NC')
temp19_df['city'] = 'Raleigh'
temp20_df = job_db('ios+developer', 'Atlanta', 'GA')
temp20_df['city'] = 'Atlanta'

ios_developer_df = pd.concat([temp1_df, temp2_df, temp3_df, temp4_df, temp5_df,
                              temp6_df, temp7_df, temp8_df, temp9_df, temp10_df,
                              temp11_df, temp12_df, temp13_df, temp14_df, temp15_df,
                              temp16_df, temp17_df, temp18_df, temp19_df, temp20_df], ignore_index=True)
print(ios_developer_df.shape)

759 ios+developer San+Jose
465 ios+developer San+Francisco
483 ios+developer Seattle
351 ios+developer Washington
317 ios+developer New+York
83 ios+developer Baltimore
107 ios+developer Boulder
90 ios+developer San+Diego
132 ios+developer Denver
15 ios+developer Huntsville
6 ios+developer Colorado+Springs
52 ios+developer Houston
24 ios+developer Trenton
154 ios+developer Dallas
28 ios+developer Columbus
175 ios+developer Austin
86 ios+developer Philadelphia
48 ios+developer Durham
47 ios+developer Raleigh
134 ios+developer Atlanta
(3224, 6)


In [0]:
data_scientist_df['job'] = 'data scientist'
web_developer_df['job'] = 'web developer'
ux_designer_df['job'] = 'ux designer'
ios_developer_df['job'] = 'ios developer'
indeed_df = pd.concat([data_scientist_df, web_developer_df, ux_designer_df, ios_developer_df], ignore_index=True)
indeed_df = indeed_df.drop_duplicates(keep=False) 
indeed_df = indeed_df.reset_index(drop=True)

In [16]:
indeed_df.head()

,job_title,company,location,salary,description,city,job
0,Data Scientist (All Levels) - Santa Clara,LeanTaaS,"Santa Clara, CA 95050",None,Help build technology that saves lives!\n\nWe'...,San Jose,data scientist
1,Data Scientist,Ford Motor Company,"Palo Alto, CA",None,Data Scientist\n\n\nJob description:\n\n\nData...,San Jose,data scientist
2,"Siri - Embedded Data Scientist, Data organization",Apple,"Santa Clara Valley, CA 95014",None,"Summary\nPosted: Dec 16, 2019\nRole Number:200...",San Jose,data scientist
3,Data Scientist,California State University,"San Jose, CA",None,The Data Scientist will leverage data that has...,San Jose,data scientist
4,Data Scientist,Spry Health,"Palo Alto, CA","$100,000 - $135,000 a year",Spry Health’s mission is to build the world’s ...,San Jose,data scientist


In [17]:
def convert_salary(sal_str, s_flag):
    
    sal_split = re.findall(r'\d+', sal_str.replace(",", ""))
    
    if len(sal_split) == 2:
        low_salary = int(sal_split[0])
        high_salary = int(sal_split[1])
    else:
        low_salary = None
        high_salary = None
    
    if s_flag == 'l':
        salary = low_salary
    else:
        salary = high_salary
    
    return salary

indeed_df['low_salary'] = indeed_df.apply(lambda x: convert_salary(x['salary'], 'l'), axis=1)
indeed_df['high_salary'] = indeed_df.apply(lambda x: convert_salary(x['salary'], 'h'), axis=1)
indeed_df = indeed_df.drop(columns=['salary'])

indeed_df.head()                                      

,job_title,company,location,description,city,job,low_salary,high_salary
0,Data Scientist (All Levels) - Santa Clara,LeanTaaS,"Santa Clara, CA 95050",Help build technology that saves lives!\n\nWe'...,San Jose,data scientist,NaN,NaN
1,Data Scientist,Ford Motor Company,"Palo Alto, CA",Data Scientist\n\n\nJob description:\n\n\nData...,San Jose,data scientist,NaN,NaN
2,"Siri - Embedded Data Scientist, Data organization",Apple,"Santa Clara Valley, CA 95014","Summary\nPosted: Dec 16, 2019\nRole Number:200...",San Jose,data scientist,NaN,NaN
3,Data Scientist,California State University,"San Jose, CA",The Data Scientist will leverage data that has...,San Jose,data scientist,NaN,NaN
4,Data Scientist,Spry Health,"Palo Alto, CA",Spry Health’s mission is to build the world’s ...,San Jose,data scientist,100000.0,135000.0


In [18]:
indeed_df.tail()

,job_title,company,location,description,city,job,low_salary,high_salary
4945,Control Systems Programmer,Ford AV,"Atlanta, GA","Compensation: Based Upon Experience, Benefits,...",Atlanta,ios developer,NaN,NaN
4946,Software Engineer,ASSA ABLOY Opening Solutions,"Marietta, GA",HID Global powers the trusted identities of th...,Atlanta,ios developer,NaN,NaN
4947,Software Engineer,Assa Abloy,"Marietta, GA",HID Global powers the trusted identities of th...,Atlanta,ios developer,NaN,NaN
4948,Training Consultant,MicroStrategy,"Atlanta, GA 30319",Job Description\n\nThe Role:\nThe Training Con...,Atlanta,ios developer,NaN,NaN
4949,Senior Software Engineer,Omada Health,"Atlanta, GA",Omada Health is on a mission to inspire and en...,Atlanta,ios developer,NaN,NaN


In [19]:
count_df.head()

,job_count,title_name,city_name


In [0]:
def clean_text(text):
    text = text.replace('\\n', ' ')               # remove newline
    text = BeautifulSoup(text, "lxml").get_text() # remove html
    text = text.replace('/', ' ')                 # remove forward slashes
    text = re.sub(r'[^a-zA-Z ^0-9]', '', text)    # letters and numbers only
    text = text.lower()                           # lower case
    text = re.sub(r'(x.[0-9])', '', text)         # remove special characters
    return text

#indeed_df['description'] = indeed_df.apply(lambda x: clean_text(x['description']), axis=1)

#indeed_df.head()

In [21]:
indeed_df['description'][0]

"Help build technology that saves lives!\n\nWe're a fast growing healthcare tech company that uses data science and machine learning to make our hospitals more efficient.\n\n\nMore than 65 hospital systems including some of the nation's largest rely on our products to improve patient access and lower wait times.\nWe've raised $70+ million dollars from prominent investors in healthcare.\nOur team includes veteran executives and the brightest minds from Google, McKinsey, Stanford, MIT, Duke, Berkeley, UIUC, and more.\nCBInsights recently named us among the top 100 AI companies in the world.\n\nWe're looking for extraordinary data scientists for our Santa Clara, California office. The Data Scientist role involves working on all stages of the data science pipeline, from acquiring and munging data, selecting appropriate models and algorithms and/or deriving custom algorithms, to testing and evaluating these algorithms and models and incorporating them into commercial products. She/He will w

In [22]:
indeed_df['description'][4]

'Spry Health’s mission is to build the world’s largest physiological dataset to map human health, understand disease, and create new medical knowledge. We are innovating in the field of physiological signal processing and are looking for an experienced and motivated researcher to join our talented R&D team. The Research Engineer will take big responsibilities on our core science team and break new ground in health monitoring technology through physiological signal processing. We are the first to collect continuous physiological signals from thousands of patients.\n\nThis is an opportunity to step into a highly innovative healthcare startup and make real-world impact. Our dataset contains important information that would be a building block in medicine and physiological monitoring, discovering new physiological biomarkers that will help us combat disease. Working with physiological signals from users who are going about their daily life is very challenging—we have made significant progr

In [0]:
# convert to sqlite3
conn = sqlite3.connect('techsearch.sqlite3')
curs = conn.cursor()
curs.execute('drop table if exists listings')
curs.execute('drop table if exists counts')
indeed_df.to_sql('listings', con=conn)
count_df.to_sql('counts', con=conn)